In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import pandas as pd
import re
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.image as mp_img
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import interp1d

import holoviews as hv
hv.extension('bokeh')

def get_colors(inp, colormap, vmin=None, vmax=None):
    norm = plt.Normalize(vmin, vmax)
    return colormap(norm(inp))

In [ ]:
datapath  = '/Volumes/extreme/repos/lab-data/characterization/'
sampletag = 'SH-00001'

imgfile = '/Volumes/extreme/repos/lab-data/characterization/photo/SH-00001/bw/dry_DSC08702.jpg'
tgafile = '/Volumes/extreme/repos/lab-data/characterization/tga/SH-00001/_analysis/peaks-onsets-mass.xlsx'
velfile = '/Volumes/extreme/repos/lab-data/characterization/autoscan/SH-00001/_analysis/elastic_before.csv'
elsfile = '/Volumes/extreme/repos/lab-data/characterization/autoscan/SH-00001/_analysis/map_Estar_before.csv'
perfile = '/Volumes/extreme/repos/lab-data/characterization/autoscan/SH-00001/_analysis/map_Permeability_before.csv'
tocfile = '/Volumes/extreme/repos/lab-data/characterization/toc/SH-00001/toc-before.xlsx'

In [ ]:
data = {}

# read the image file
data['img'] = mp_img.imread(imgfile)

# read the estar map
data['els'] = pd.read_csv(elsfile, usecols=range(0,4))

# read velocity map
data['vel'] = pd.read_csv(velfile, usecols=range(0,8))

# read the permability file
data['per'] = pd.read_csv(perfile, usecols=range(0,4))

# read the tga peaks data
data['tga'] = pd.read_excel(tgafile, usecols=[1,2,3])

# read the toc file
data['toc'] = pd.read_excel(tocfile, usecols=range(6,12))

measkeys = [key for key in data.keys()][1:]
askeys   = measkeys[:3]
for key in measkeys:
    data[key].columns = [re.sub(r'\W+', '', c) for c in data[key].columns]
    colnames = data[key].columns.values
    colnames[0] = 'i'
    data[key].columns = colnames

data['vel'] = data['vel'].drop(['vp','vs'], axis=1)

In [ ]:
descriptions = pd.DataFrame(columns=['probe', 'var', 'vmin', 'vmax'])
kk = []
nrows = 0
for key in measkeys:
    colnames = data[key].columns.values

    if key in askeys:
        colnames = colnames[3:]
        nrowsnew = data[key].shape[0]
        if nrowsnew>nrows:
            nrows = nrowsnew
    else:
        colnames = colnames[1:]
    for c in colnames:
        kk.append([key, c, data[key].loc[:,c].min(), data[key].loc[:,c].max()])

for row in kk:
    descriptions.loc[len(descriptions)] = row

asrowsmax = nrows

del colnames, kk, nrowsnew, nrows

In [ ]:
varsdict = {
    'Estar':{'units':'E* (GPa)', 'multiplier':1e9},
    'vp':{'units':'v (m/s)', 'multiplier':1},
    'vs':{'units':'v (m/s)', 'multiplier':1},
    'E':{'units':'E (GPa)', 'multiplier':1e9},
    'K':{'units':'K (GPa)', 'multiplier':1e9},
    'nu':{'units':r'$\nu$', 'multiplier':1},
    'Onset':{'units':'deg C', 'multiplier':1},
    'Masschange':{'units':r'$\mu g/V$', 'multiplier':1},
    'Permeability':{'units':r'$\mu_d$ (mD)'},
    'PI':{'units':r'%'}
}

In [ ]:
asrowsmax
# # vardata = {'els':{'var':['Estar'],'dsc':['E* (GPa)']}, 'vel':{'var':['E','K','nu'],'dsc':['E (GPa)','K (GPa)','$nu $']}, 'per':{'var':['Permeability'], 'dsc':['$\mu$ (mD)']}}
# # vardata['extremes'] = {}
# for key in measkeys:
#     colnames = data[key].columns.values
#     if key in askeys:
#         colnames = colnames[3:]
#     else:
#         colnames = colnames[1:]
    
#     print(colnames)
# #     for skey in :
# #     varsdict[key]['extremes'] = [0,0]
# # print(varsdict)
# #     for j, skey in enumerate(vardata[key]['var']):
# #         print(skey, vardata[key]['dsc'][j], sep='\t')
# #         vardata['extremes'][skey] = [data[key][vardata[key]['var'][j]].max(), data[key][vardata[key]['var'][j]].min()]

In [ ]:
for key in askeys:
    colnames = data[key].columns.values
    df = pd.DataFrame(np.zeros([asrowsmax,colnames.size]), columns=colnames)
    df.iloc[:,1:] = np.nan
    nindex = df.index.values[data[key].iloc[:,0]]
    for i in nindex:
        df.iloc[i,0] = 1
        df.iloc[i,1:] = data[key].loc[data[key].iloc[:,0]==i,data[key].columns.values[1:]].values
    data[key] = df

In [ ]:
for key in data.keys():
    print(key, data[key].shape, sep='\t')

In [ ]:
#data['vel'].loc[data['vel'].iloc[:,0]==0]

In [ ]:
xomin = data['per']['x'].min(axis=0)
xmax = data['per']['x'].max(axis=0)
yomin = data['per']['y'].min(axis=0)
ymax = data['per']['y'].max(axis=0)

xmin = ymin = 0
xmax = xmax - xomin
ymax = ymax - yomin

print(xmin, xmax, ymin, ymax)

In [ ]:
# build x and y elements for tga and toc
bwidth = {}
for key in ['tga', 'toc']:
    bwidth[key] = (ymax - ymin)/data[key].shape[0]
    data[key]['x']  = (xmax - xmin)/2.
    data[key]['y']  = np.linspace(ymin, ymax, data[key].shape[0])  
    data[key]['yc'] = np.linspace(ymin+bwidth[key]/2, ymax-bwidth[key]/2, data[key].shape[0])  

In [ ]:
xygrid = {}
for key in askeys:
    xg = np.unique(data[key]['x'].values - xomin)
    yg = np.unique(data[key]['y'].values - yomin)
    xg = xg[xg>=0.]
    yg = yg[yg>=0.]
    xygrid[key] = np.meshgrid(xg, yg)
#     print(xygrid[key][0].shape, xg.shape[0]*yg.shape[0], data[key].shape)

In [ ]:
# colors = get_colors(data[key]['TOC'].values, cm.viridis, vmin = data[key]['TOC'].min(), vmax = data[key]['TOC'].max())


In [ ]:
# bwidth[key]
# fig = plt.figure(figsize=(3,15))
# ax = fig.add_subplot(111)
# p = ax.barh(data[key]['yc'].values,data[key]['TOC'].values, bwidth[key]*0.8, align='center', color=colors)

In [ ]:
# fig = plt.figure(figsize=(3,15))
# ax = fig.add_subplot(111)

# xs,ys = xygrid[key][0].shape
# gg = np.reshape(data[key].iloc[:,-1].values, (xs,ys))

# ax.pcolor(xygrid[key][0], xygrid[key][1], gg, cmap='viridis', vmin=gg.min(), vmax=gg.max())
vardata['extremes']

In [ ]:
plt.rcParams.update({'font.size': 12})
fig, axs = plt.subplots(nrows=1, ncols=9, sharey=True, sharex=False, figsize=(35,25))
fig.subplots_adjust(wspace=0.3)#, left=0, right=1)

ax = axs[0]
pc = ax.imshow(data['img'], interpolation = 'gaussian', extent=[xmin,xmax,ymin,ymax])
ax.set_ylabel('y (mm)')
ax.set_xlabel('x (mm)')
cb = fig.colorbar(pc, ax=ax, orientation="horizontal", pad=0.05) #orientation="horizontal", pad=0.2
cb.set_label('')
cb.remove()

i = 1
for key in askeys:
    xs,ys = xygrid[key][0].shape
    cvars = descriptions.loc[descriptions['probe']==key,['var','vmin','vmax']].copy()
    #print(cvars['var'].values)
    for c in range(len(cvars)):
        var  = cvars.iloc[c,0]
        vmin, vmax = cvars.iloc[c,1:].values
        # print(var, vmin, vmax, sep='\t')
        gg = np.reshape(data[key].loc[:,var].values, (xs,ys))
        
        ax = axs[i]
        pc = ax.pcolor(xygrid[key][0], xygrid[key][1], gg, cmap='viridis', vmin=vmin, vmax=vmax)
        
        ax.title.set_text(var)
        ax.set_xlabel('x (mm)')
        
        cb = fig.colorbar(pc, ax=ax, orientation="horizontal", pad=0.05, ticks=[vmin, vmax]) #orientation="horizontal", pad=0.2
        cb.set_label(varsdict[var]['units'])
        i += 1

for key in ['tga','toc']:
    print(bwidth[key])
    cvars = descriptions.loc[descriptions['probe']==key,['var','vmin','vmax']].copy()
    #print(cvars['var'].values)
    for c in range(len(cvars)):
        var  = cvars.iloc[c,0]
        print(var)
        if var in ['Onset', 'Masschange', 'TOC', 'PI']:
            vmin, vmax = cvars.iloc[c,1:].values
            x = c = data[key][var].values
            y = data[key]['yc'].values
            colors = get_colors(data[key][var].values, cm.viridis, vmin = vmin, vmax = vmax)
            #print(var, vmin, vmax, sep='\t')
            ax = axs[i]
            pc = ax.scatter(x, y, c=c, cmap='viridis')#, bwidth[key]*0.8, align='center', color=colors)
            ax.title.set_text(var)
            ax.set_xlabel(varsdict[var]['units'])
            
            x_new = np.linspace(y.min(), y.max(),500)

            f = interp1d(y, x, kind='quadratic')
            y_smooth=f(x_new)

            ax.plot (y_smooth,x_new)
            
            i +=1
        
            cb = fig.colorbar(pc, ax=ax, orientation="horizontal", pad=0.05, ticks=[vmin, vmax]) #orientation="horizontal", pad=0.2
            cb.set_label(varsdict[var]['units'])

In [ ]:
cvars['var']

In [ ]:

from matplotlib import pyplot as plt

img = cv2.imread('images/flicker2.jpg',0)
f = np.fft.fft2(img)
fshift = np.fft.fftshift(f)
# calculate amplitude spectrum
mag_spec = 20*np.log(np.abs(fshift))

r = f.shape[0]/2        # number of rows/2
c = f.shape[1]/2        # number of columns/2   
p = 3                         
n = 1                   # to suppress all except for the DC component       
fshift2 = np.copy(fshift)

# suppress upper part
fshift2[0:r-n , c-p:c+p] = 0.001
# suppress lower part
fshift2[r+n:r+r, c-p:c+p] = 0.001
# calculate new amplitude spectrum
mag_spec2 = 20*np.log(np.abs(fshift2))
inv_fshift = np.fft.ifftshift(fshift2)
# reconstruct image
img_recon = np.real(np.fft.ifft2(inv_fshift))

plt.subplot(131),plt.imshow(img, cmap = 'gray')
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(132),plt.imshow(mag_spec, cmap = 'gray')
plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])
plt.subplot(133),plt.imshow(mag_spec2, cmap = 'gray')
plt.title('Magnitude Spectrum after suppression'), plt.xticks([]), plt.yticks([])
plt.show()

plt.figure()
plt.subplot(121),plt.imshow(img, cmap = 'gray')
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(img_recon, cmap = 'gray')
plt.title('Output Image'), plt.xticks([]), plt.yticks([])
plt.show()